In [2]:
## 발행정보 - To IRS FRN leg
spread = 0.0    # 쿠폰에 더해지는 스프레드

reset_dates = ['2021-11-30','2022-02-28','2022-05-31','2022-08-31','2022-11-30','2023-02-28','2023-05-31','2023-08-31']
begin_dates = ['2021-12-01','2022-03-02','2022-06-02','2022-09-01','2022-12-01','2023-03-02','2023-06-01','2023-09-01']
end_dates = ['2022-03-02','2022-06-02','2022-09-01','2022-12-01','2023-03-02','2023-06-01','2023-09-01','2023-12-01']
settle_dates = ['2022-03-02','2022-06-02','2022-09-01','2022-12-01','2023-03-02','2023-06-01','2023-09-01','2023-12-01']

fixing_dates = ['2021-11-30']   # 금리 결정일 <= 평가일 <= 쿠폰지급일 인 경우 결제가 되지 않았기 때문에 사용
fixing_prices = [0.0126]         # 위와 같은 상황에서 금리값

## 마켓데이터 - 2021-11-05일자 IRSKRW_SPOT Curve
spot_tenors =[0.00278,0.25,0.5,0.75,1,1.25,1.5,1.75,2,2.25,2.5,2.75,3]
spot_rates = [0.0104,0.01258,0.013654,0.014303,0.01514,0.01578,0.016408,0.016746,0.017085,0.017248,0.017411,0.017575,0.017738]

#계산일(평가일)
val_date = '2021-11-30'      

In [3]:
import datetime
import math

# 사용 할 함수 정의

# 보간법을 이용하여 지정된  tenor 사이의 teminate_time 의 이자율 산출
def get_ir_interpolation(target_tenors, target_rates, selected_tenor):
    if len(target_tenors) == 1:
        return target_rates[0]
    if target_tenors[-1] <= selected_tenor:
        return target_rates[:-1]

    for i, tenor in enumerate(target_tenors):
        if tenor == selected_tenor:
            return target_rates[i]
        else:
            if tenor > selected_tenor:
                return target_rates[i-1] + (selected_tenor - target_tenors[i-1]) \
                       / (target_tenors[i] - target_tenors[i-1]) * (target_rates[i] - target_rates[i-1])


In [4]:
# 두 날짜 차이의 연환산값 산출
def get_year_diff_two_dates(start_date, end_date):
    start = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.datetime.strptime(end_date, "%Y-%m-%d")
    return ((end - start).days) / 365

# 단순히 두 날짜의 차이 산출
def get_diff_two_dates(start_date, end_date):
    start = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.datetime.strptime(end_date, "%Y-%m-%d")
    return ((end - start).days)

In [5]:
# 평가일(계산일) ~ 각 쿠폰일의 날짜 차이에 따른 할인 금리 만들기
discount_tenors = []
discount_rates = []

for date in settle_dates:
    discount_tenors.append(get_year_diff_two_dates(val_date,date))


In [6]:
for settle_date, tenor in zip(settle_dates, discount_tenors):
    print(f"결제일 {settle_date} ~ 평가일 {val_date} 의 연 환산 값 : {round(tenor, 10)}")

결제일 2022-03-02 ~ 평가일 2021-11-30 의 연 환산 값 : 0.2520547945
결제일 2022-06-02 ~ 평가일 2021-11-30 의 연 환산 값 : 0.504109589
결제일 2022-09-01 ~ 평가일 2021-11-30 의 연 환산 값 : 0.7534246575
결제일 2022-12-01 ~ 평가일 2021-11-30 의 연 환산 값 : 1.002739726
결제일 2023-03-02 ~ 평가일 2021-11-30 의 연 환산 값 : 1.2520547945
결제일 2023-06-01 ~ 평가일 2021-11-30 의 연 환산 값 : 1.501369863
결제일 2023-09-01 ~ 평가일 2021-11-30 의 연 환산 값 : 1.7534246575
결제일 2023-12-01 ~ 평가일 2021-11-30 의 연 환산 값 : 2.002739726


In [7]:
# 위에서 구한 discount_tenors 값에 따른 각각의 이자율 산출
for tenor in discount_tenors:
    discount_rates.append(get_ir_interpolation(target_tenors=spot_tenors, target_rates=spot_rates, selected_tenor=tenor))

In [8]:
for tenor, rate in zip(discount_tenors, discount_rates):
    print(f"tenor {round(tenor, 7)} 의 rate : {round(rate, 7)}")

tenor 0.2520548 의 rate : 0.0125888
tenor 0.5041096 의 rate : 0.0136647
tenor 0.7534247 의 rate : 0.0143145
tenor 1.0027397 의 rate : 0.015147
tenor 1.2520548 의 rate : 0.0157852
tenor 1.5013699 의 rate : 0.0164099
tenor 1.7534247 의 rate : 0.0167506
tenor 2.0027397 의 rate : 0.0170868


* Discount Tenor 산출 공식 : Discount Factor $= exp(-rate * tenor)$


In [9]:
# 엑셀 시트 ANS 탭의 DiscountFactor부분
discount_factor = []
for tenor, rate in zip(discount_tenors, discount_rates):
    discount_factor.append(math.exp(- rate * tenor))

In [30]:
for dis_tenor in discount_tenors:
    print(dis_tenor)

0.25205479452054796
0.5041095890410959
0.7534246575342466
1.0027397260273974
1.252054794520548
1.5013698630136987
1.7534246575342465
2.0027397260273974


In [10]:
for i, df in enumerate(discount_factor):
    print(f"{i} - th discount factor : {df}")

0 - th discount factor : 0.9968319545769854
1 - th discount factor : 0.9931351808473751
2 - th discount factor : 0.9892730767596721
3 - th discount factor : 0.9849262513230784
4 - th discount factor : 0.9804301379687609
5 - th discount factor : 0.9756637627228489
6 - th discount factor : 0.9710561436388943
7 - th discount factor : 0.9663585095558931


In [9]:
# forward rate 산출
# 각각의 선도 이자율의 의미 : 평가일 시점에서 파악한 i번째 기산일 ~ i번째 기말일까지의 이자율

In [11]:
coupon_list = []
coupon_num = len(settle_dates)

for i in range(coupon_num):
    if get_year_diff_two_dates(val_date, settle_dates[i]) <= 0:    # 쿠폰지급일 <= 평가일 인 경우 쿠폰 0
        coupon_list.append(0) 
    elif get_year_diff_two_dates(reset_dates[i], val_date) >= 0:   # 리셋일 <= 평가일 <= 쿠폰지급일 인 경우 이미 리셋일에 정해진 이자가 fixing_prices에 있기 때문에 그것을 그대로 가져옴
        idx = fixing_dates.index(reset_dates[i])
        coupon_list.append(fixing_prices[idx])
    else:                                                             # 평가일 < 리셋일 인 경우 Forward Rate 를 산출
        coupon_list.append((discount_factor[i-1] / discount_factor[i] - 1) 
            / (get_diff_two_dates(settle_dates[i-1], settle_dates[i]) / 365))


In [12]:
# 엑셀 시트 ANS 탭의 확정(추정)금리 부분
# 각각의 금리는 연 환산이므로 실제로 지급하는 쿠폰은 "기말일-기산일" 로 나눠주어야 한다. 
print("각 쿠폰일의 추정 금리")
for i, fr in enumerate(coupon_list):
    print(f"{i} - th forward rate : {fr}")    

각 쿠폰일의 추정 금리
0 - th forward rate : 0.0126
1 - th forward rate : 0.014767927097782805
2 - th forward rate : 0.015658828102157914
3 - th forward rate : 0.01770190309725581
4 - th forward rate : 0.01839382589124703
5 - th forward rate : 0.019594740999962446
6 - th forward rate : 0.01882509884436375
7 - th forward rate : 0.01949810396270438


In [13]:
# 확정(추정)금리가 연이자율이기 때문에 기산일~기말일 사이로 바꿔주기 위해서 곱할 것들을 만들어줌
begin_end_diff = []
for begin, end in zip(begin_dates, end_dates):
    begin_end_diff.append(get_year_diff_two_dates(begin, end))

In [14]:
# 엑셀 시트 ANS 탭의 지급 쿠폰 부분
pay_cpn = []
for i in range(len(coupon_list)):
    pay_cpn.append(0)

for i, coupon in enumerate(coupon_list):
    if coupon == 0:
        pay_cpn[i] = 0
    else:
        pay_cpn[i] = (coupon + spread) * begin_end_diff[i]    # 연이율에서 기산일-기말일 로 바꿔줌


In [15]:
for i, act_cpn_rate in enumerate(pay_cpn):
    print(f"{i} - th Pay Coupon rate : {act_cpn_rate} at settle date {settle_dates[i]}")

0 - th Pay Coupon rate : 0.003141369863013699 at settle date 2022-03-02
1 - th Pay Coupon rate : 0.003722326830126077 at settle date 2022-06-02
2 - th Pay Coupon rate : 0.003903981800811973 at settle date 2022-09-01
3 - th Pay Coupon rate : 0.004413351183151448 at settle date 2022-12-01
4 - th Pay Coupon rate : 0.004585857961927342 at settle date 2023-03-02
5 - th Pay Coupon rate : 0.004885264194511185 at settle date 2023-06-01
6 - th Pay Coupon rate : 0.00474495642104511 at settle date 2023-09-01
7 - th Pay Coupon rate : 0.004861171124948216 at settle date 2023-12-01


In [16]:
result = 0
for i in range(len(pay_cpn)):
    if i < (len(pay_cpn) -1):
        result += pay_cpn[i] * discount_factor[i]
    else:
        result += (pay_cpn[i]) * discount_factor[i]

In [19]:
print(f"Floating Leg Price = {result}")

Floating Leg Price = 0.03360486288168947


In [20]:
fixed_rate = 0.0171

# Floating Leg 의 쿠폰 쿠폰 스케쥴과 동일 하기 때문에 DF를 그대로 사용

In [21]:
fixed_coupon_list = []
coupon_num = len(settle_dates)

for i in range(coupon_num):
    if get_year_diff_two_dates(val_date, settle_dates[i]) <= 0:    # 쿠폰지급일 <= 평가일 인 경우 쿠폰 0
        fixed_coupon_list.append(0) 
    else:                                                             # 평가일 < 리셋일 인 경우 Forward Rate 를 산출
        fixed_coupon_list.append(fixed_rate)

In [23]:
# 확정(추정)금리가 연이자율이기 때문에 기산일~기말일 사이로 바꿔주기 위해서 곱할 것들을 만들어줌
begin_end_diff = []
for begin, end in zip(begin_dates, end_dates):
    begin_end_diff.append(get_year_diff_two_dates(begin, end))

In [24]:
# 엑셀 시트 ANS 탭의 지급 쿠폰 부분
fixed_pay_cpn = []
for i in range(len(fixed_coupon_list)):
    fixed_pay_cpn.append(0)

for i, coupon in enumerate(fixed_coupon_list):
    if coupon == 0:
        fixed_pay_cpn[i] = 0
    else:
        fixed_pay_cpn[i] = (coupon) * begin_end_diff[i] 

In [26]:
fixed_result = 0
for i in range(len(pay_cpn)):
    if i < (len(pay_cpn) -1):
        fixed_result += fixed_pay_cpn[i] * discount_factor[i]
    else:
        fixed_result += (fixed_pay_cpn[i]) * discount_factor[i]

print(f"Fixed Leg Price = {fixed_result}")

Fixed Leg Price = 0.03359155004442398


In [29]:
print(f"IRS Price = {result - fixed_result}")

IRS Price = 1.3312837265487543e-05
